In [1]:
import altair as alt
import pandas as pd
import numpy as np
#get the data
from vega_datasets import data

[source](https://medium.com/analytics-vidhya/interactive-data-viz-using-altair-873139771fe2) 

In [ ]:
settle_data = data.seattle_weather()
settle_data.head()
source.head()

In [ ]:
settle_data.weather.unique()

In [ ]:
settle_data.date.unique()

In [ ]:
## Add brush and click

In [14]:
# Defining the color scale for the plot (distribution based on Weather types)
scale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                  range=['#e7ba52', '#a7a7a7', '#aec7e8', '#1f77b4', '#9467bd'])
color = alt.Color('weather:N', scale=scale) 
# N : used to specify nominal data

##############################################################################
### Add brush and click
# We create two selections
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
##############################################################################
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

- `T`: Specify Time data
- `Q`: Specify quantitative data
- `O` | Specify ordinal data

## Read Banana

In [10]:
# colnames=['block', 'banana_number', 'treatment', 'day', 'humidity', 'temperature', 'black_ratio'] 
b_data = pd.read_csv("db.csv")
b_data.head()
b_data.tail()

,Unnamed: 0,y,block,banana_number,treatment,day,humidity,temperature,black_ratio,weight
155,156,0.061001,B,8,control,9,48,70,0.634690,777
156,157,0.080037,B,1,control,10,46,73,0.653726,130
157,158,0.080037,B,5,control,10,46,73,0.653726,116
158,159,0.080037,B,6,control,10,46,73,0.653726,122
159,160,0.080037,B,8,control,10,46,73,0.653726,124


- `T`: Specify Time data
- `Q`: Specify quantitative data
- `O` | Specify ordinal data

In [11]:
b_data.head()

,Unnamed: 0,y,block,banana_number,treatment,day,humidity,temperature,black_ratio,weight
0,1,-0.037323,N,1,control,1,37,72,0.858422,171
1,2,-0.049106,N,3,treatment,1,37,72,0.782818,172
2,3,-0.036415,N,7,treatment,1,37,72,0.788474,168
3,4,-0.037093,N,5,control,1,37,72,0.863295,165
4,5,-0.055513,N,2,treatment,1,37,72,0.790132,160


In [29]:
brush = alt.selection(type='interval')

points = alt.Chart(b_data).mark_circle().encode(
    alt.X('day:Q', title='Date'),
    alt.Y('y:Q',
         title='Normalized changes in HSV',
         scale=alt.Scale(domain=[-0.1, 0.15])
       ),
#     size= 'subject:N', #should be the weight
    color= alt.condition(brush, 'treatment:N', alt.value('lightgray')),
#     size = 'subject:N',
#     color= alt.condition(brush, 'treatment:N', alt.value('lightgray'),legend=alt.Legend(orient="right"))
).add_selection(
    brush
).transform_filter(
    click
)

temp = alt.Chart(b_data).mark_circle().encode(
    alt.X('humidity:Q', title='Date',
          scale=alt.Scale(domain=[30, 60])),
    alt.Y('temperature:Q',
         title='Maximum Daily Temperature (C)',
         scale=alt.Scale(domain=[65, 90])
       ),
#     size= 'subject:N', #should be the weight
    color= alt.condition(brush, 'treatment:N', alt.value('lightgray'),legend=alt.Legend(orient="top"))
).transform_filter(
    brush
).add_selection(
    click
)


points | temp

alt.HConcatChart(...)

In [ ]:
np.random.seed(0)

n_objects = 20
n_times = 50

# Create one (x, y) pair of metadata per object
locations = pd.DataFrame({
    'id': range(n_objects),
    'x': np.random.randn(n_objects),
    'y': np.random.randn(n_objects)
})

locations.head()

In [ ]:
# Create a 50-element time-series for each object
timeseries = pd.DataFrame(np.random.randn(n_times, n_objects).cumsum(0),
                          columns=locations['id'],
                          index=pd.RangeIndex(0, n_times, name='time'))

timeseries.head()

In [ ]:
# Melt the wide-form timeseries into a long-form view
timeseries = timeseries.reset_index().melt('time')
timeseries.head()

In [ ]:
# Merge the (x, y) metadata into the long-form view
timeseries['id'] = timeseries['id'].astype(int)  # make merge not complain
data = pd.merge(timeseries, locations, on='id')

In [ ]:
# Data is prepared, now make a chart

#selector = alt.selection_single(empty='all', fields=['id'])
selector = alt.selection_single(fields=['id'])

base = alt.Chart(data).properties(
    width=250,
    height=250
).add_selection(selector)



points = base.mark_point(filled=True, size=200).encode(
    x='mean(x)',
    y='mean(y)',
    color=alt.condition(selector, 'id:O', alt.value('lightgray'), legend=None),
)

timeseries = base.mark_line().encode(
    x='time',
    y=alt.Y('value', scale=alt.Scale(domain=(-15, 15))),
    color=alt.Color('id:O', legend=None)
).transform_filter(
    selector
)

points | timeseries